## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Предметом исследования являются показатели платежеспособности клиента: доход, семейное положение,количество детей клиента, цели получения кредита.

Цель работы: исследование надёжности заёмщиков, т.е. изучение тех факторов, которые могут повлиять на погашение кредита в срок. Для этого понадобится: 
1. Изучить данные; 
2. Обработать полученные данные (выявить проблемы); 
3. Определить существует ли зависимость между наличием детей и возвратом кредита в срок; 
4. Определить существет ли зависимость между семейным положением и возвратом кредита в срок; 
5. Определить существует ли зависимость между уровнем дохода и возвратом кредита в срок; 
6. Определить, как разные цели кредита влияют на его возврат в срок; 
7. Сделать выводы по исследованию надежности заемщиков. 
 


### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных таблицы df.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.

Получаем перечень названий столбцов.

In [3]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

### Вывод

Каждая строка таблицы содержит информацию о клиентах, его трудовом стаже и суммарном доходе. Также видим цели получения кредита.Проблемы, которые нужно решать: 
1 Пропуски в трудовом стаже работы days_employed и в суммарном доходе total_income;
2 Дубликаты в образовании education, в цели получения кредита purpose;
3 Артефакты - отрицательные значения в стаже работы days_employed.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
# Проверим данные на наличие пропусков:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах 'days_employed' и 'total_income' одинаковое количество пропусков. Можно предположить, что пропуски в одних и тех же строках. Проверим каждая ли строка со стажем NaN имеет значение дохода:

In [5]:
#Отсортируем таблицу по столбцу 'days_employed'
df_sort_days_employed = df.sort_values(by = 'days_employed',ascending = False)
df_sort_days_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [6]:
#Посчитаем количество строк с пропусками в 'days_employed' и 'total_income'
count = df_sort_days_employed[df_sort_days_employed['days_employed'].isnull() & df_sort_days_employed['total_income'].isnull()].count()
count

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Пустые значения свидетельствуют, что доступна не вся информация по трудовому стажу и доходу. Причем, где отсутвуют данные по стажу, там же нет данных и по доходу. Значит существет какая-то зависимость между этими столбцами. В обоих столбцах вещественные значения, можно предположить, что они получены вычислениями. Скажем, в столбце 'total_income' данные могли быть получены, как средний ежемесячный доход. В столбце 'days_employed' отрицательные значения могли появиться, как ошибки выгрузки и значения больше похожи на трудовой стаж в годах.

In [7]:
#Переведем трудовой стаж в года и уберем минусовые значения:
df['years_employed'] = abs(df['days_employed'] / 365)
df.drop(['days_employed'], axis='columns', inplace=True)
df.head(50)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932.235814
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,2.537495
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7.888225
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0.418574
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,18.985932
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,5.996593


Видим, что некоторый стаж получился в 1000+ лет - что явно много. Можно предположить, что такой аномально большой стаж дан в часах, а не днях. Причем он наблюдается только у пенсионеров. Для анализа разделим аномальные значения в  стаже 'years_employed' на 24. А затем пустые значения в стаже заменим на медианные.

In [8]:
# Функция заменяет 'years_employed' на 'years_employed' / 24, если стаж >100
def years_employed_hour(years_employed):
    if years_employed > 100:
        return years_employed / 24
    else:
        return years_employed

df['years_employed'] = df['years_employed'].apply(years_employed_hour) #применяем функцию 'years_empolyed' к столбцам 
years_employed_median = df['years_employed'].median() # находим среднее
df['years_employed'] = df['years_employed'].fillna(years_employed_median) # заменяем все пустые сроки в стаже на медиану
df.head(50)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,38.843159
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,2.537495
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7.888225
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0.418574
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,18.985932
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,5.996593


Заполним пропуски в доходе 'total_income' по подгруппам 'income_type' - в зависимости от рода деятельности.

In [9]:
#По каждой подгруппе посмотрим количество клиентов и среднее значение дохода.
income_type_mean = df.groupby('income_type')['total_income'].agg(['count', 'mean']).sort_values(by =  'count', ascending = False)
income_type_mean

,count,mean
income_type,,
сотрудник,10014,161380.260488
компаньон,4577,202417.461462
пенсионер,3443,137127.465690
госслужащий,1312,170898.309923
безработный,2,131339.751676
в декрете,1,53829.130729
предприниматель,1,499163.144947
студент,1,98201.625314


Т.к. количество клиентов в декрете, студентов, предпринимателей, безработных минимально (1-2 человека), то удалим эти строки из таблицы, они не информативны.

In [10]:
df = df.loc[(df['income_type'] != 'безработный') & (df['income_type'] != 'в декрете') 
            & (df['income_type'] != 'предприниматель') & (df['income_type'] != 'студент')]
income_type_mean = df.groupby('income_type')['total_income'].agg(['count', 'mean']).sort_values(by =  'count', ascending = False)
income_type_mean

,count,mean
income_type,,
сотрудник,10014,161380.260488
компаньон,4577,202417.461462
пенсионер,3443,137127.465690
госслужащий,1312,170898.309923


In [11]:
#Возвращает доход по имеющемуся значению дохода 'total_income' и рода деятельности 'income_type', используя правила:
import math
def total_income_mean(row):
    total_income = row['total_income']
    income_type = row['income_type']
    if math.isnan(total_income) == False:  #Возвращает False, если total_income является NaN         
        return total_income
    elif math.isnan(total_income) == True:   #во всех остальных случаях возвращается True
        if income_type == 'пенсионер': # если клиент - пенсионер
            return 137127.465690       # заменяем на среднее значение по пенсионерам
        elif income_type == 'компаньон':
            return 202417.461462
        elif income_type == 'госслужащий':
            return 170898.309923
        else:
            return 161380.260488

df['total_income'] = df.apply(total_income_mean,axis = 1)   
df.head(50)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,38.843159
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,2.537495
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7.888225
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0.418574
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,18.985932
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,5.996593


In [12]:
df.info() # выполняем проверку

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21519 entries, 0 to 21524
Data columns (total 12 columns):
children            21519 non-null int64
dob_years           21519 non-null int64
education           21519 non-null object
education_id        21519 non-null int64
family_status       21519 non-null object
family_status_id    21519 non-null int64
gender              21519 non-null object
income_type         21519 non-null object
debt                21519 non-null int64
total_income        21519 non-null float64
purpose             21519 non-null object
years_employed      21519 non-null float64
dtypes: float64(2), int64(5), object(5)
memory usage: 2.8+ MB


### Вывод

На этапе предобработки в данных обнаружились пропуски в трудовом стаже и доходе. Их удаление позволит провести анализ точнее. Причем, где отсутвапли данные по стажу, там же не было данных и по доходу. Зависимость между этими столбцами существет. В столбце days_employed отрицательные значения могли появиться, как ошибки выгрузки и значения больше похожи на трудовой стаж в годах. Поэтому их заменили на положительные и пересчитали стаж в годах. Разделили аномальные значения в  стаже years_employed на 24, предположив, что такой аномально большой стаж дан в часах. Так как сведения о клиентах важно сохранить для анализа, поэтому не просто удалили все пропущенные значения, а заполнили пропущенные значения по определенным правилам. Пустые значения в стаже years_employed заменили на медианные. Пропуски в доходе total_income заполнили по подгруппам income_type - в зависимости от рода деятельности клиента средним значением по подгруппе.

### Замена типа данных

Столбцы 'years_employed' и 'total_income' распознаны как вещественные. Они будут выглядеть аккуратнее, если их заменить целыми числами, на итоговых выводах это отразится несущественно. Заменим вещественный тип данных на целочисленный методом astype(). 

In [13]:
df['years_employed'] = df['years_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,12
21521,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,39
21522,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,5
21523,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,8


In [14]:
df.info()# проверка

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21519 entries, 0 to 21524
Data columns (total 12 columns):
children            21519 non-null int64
dob_years           21519 non-null int64
education           21519 non-null object
education_id        21519 non-null int64
family_status       21519 non-null object
family_status_id    21519 non-null int64
gender              21519 non-null object
income_type         21519 non-null object
debt                21519 non-null int64
total_income        21519 non-null int64
purpose             21519 non-null object
years_employed      21519 non-null int64
dtypes: int64(7), object(5)
memory usage: 2.8+ MB


### Вывод

Теперь в таблице 5 столбцов распознаны, как строки - тип object, а 7 - как целые числа - тип int.

### Обработка дубликатов

In [15]:
# Получаем суммарное количество дубликатов в таблице df
df.duplicated().sum() 

54

In [16]:
# удаляем все дубликаты из таблицы df специальным методом
df = df.drop_duplicates().reset_index(drop=True) 
df.duplicated().sum() # проверка

0

In [17]:
#Посчитаем количество дубликатов в столбце family_status
df['family_status'].value_counts() 

женат / замужем          12341
гражданский брак          4161
Не женат / не замужем     2809
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [18]:
#Все символы приведем к нижнему регистру:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].value_counts()

женат / замужем          12341
гражданский брак          4161
не женат / не замужем     2809
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [19]:
#Посчитаем количество дубликатов в столбце education и все символы приведем к нижнему регистру:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15186
высшее                  5247
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [20]:
#Посчитаем количество дубликатов в столбце income_type
df['income_type'].value_counts()

сотрудник      11091
компаньон       5080
пенсионер       3837
госслужащий     1457
Name: income_type, dtype: int64

In [21]:
# Получаем суммарное количество дубликатов в таблице df
df.duplicated().sum()

17

In [22]:
# удаляем все дубликаты из таблицы df специальным методом
df = df.drop_duplicates().reset_index(drop=True) 
df.duplicated().sum() # проверка

0

In [23]:
#Посчитаем количество дубликатов в столбце purpose
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья для сдачи                   650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    634
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              606
покупка жилой недвижимости                605
на покупку своего автомобиля              505
заняться высшим образованием      

In [24]:
# Сохраненим в переменной purpose список уникальных значений целей получения кредита:
purpose = df['purpose'].unique()
purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

### Вывод

На этапе предобработки в данных обнаружились не только пропуски, но и дубликаты. Строки в семейном положении и образовании отличались регистром, поэтому они переведены к нижнему регистру. Они могли появиться из за некорректного ввода данных.В целях получения кредита также видим дубликаты. Их удаление позволит провести анализ точнее. Для удаления дубликатов в целях получения кредита нужно провести лемматизацию слов.

### Лемматизация

In [25]:
# Импортируем pymystem3:
from pymystem3 import Mystem
m = Mystem() 

In [26]:
#Лемматизируем данные столбца purpose:
lemma_list=[]
for row in purpose:
   lemma_list.append(''.join(m.lemmatize(row)).strip('\n'))#с помощью join склеиваем результат для наглядности
lemma_list

['покупка жилье',
 'приобретение автомобиль',
 'дополнительный образование',
 'сыграть свадьба',
 'операция с жилье',
 'образование',
 'на проведение свадьба',
 'покупка жилье для семья',
 'покупка недвижимость',
 'покупка коммерческий недвижимость',
 'покупка жилой недвижимость',
 'строительство собственный недвижимость',
 'недвижимость',
 'строительство недвижимость',
 'на покупка подержать автомобиль',
 'на покупка свой автомобиль',
 'операция с коммерческий недвижимость',
 'строительство жилой недвижимость',
 'жилье',
 'операция со свой недвижимость',
 'автомобиль',
 'заниматься образование',
 'сделка с подержанный автомобиль',
 'получение образование',
 'автомобиль',
 'свадьба',
 'получение дополнительный образование',
 'покупка свой жилье',
 'операция с недвижимость',
 'получение высокий образование',
 'свой автомобиль',
 'сделка с автомобиль',
 'профильный образование',
 'высокий образование',
 'покупка жилье для сдача',
 'на покупка автомобиль',
 'ремонт жилье',
 'заниматься вы

### Вывод

С помощью лемматизации привели слова к его словарной форме (лемме).Теперь работать с данными будет удобнее.

### Категоризация данных

In [27]:
main = df[['children', 'years_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt','total_income', 'purpose']]
main 

,children,years_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,38,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21443,1,12,43,1,1,F,компаньон,0,224791,операции с жильем
21444,0,39,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21445,1,5,38,1,1,M,сотрудник,1,89672,недвижимость
21446,3,8,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


Создадим словарь по семейному положению:

In [28]:
family_dict = df[['family_status_id', 'family_status']]
family_dict

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
...,...,...
21443,1,гражданский брак
21444,0,женат / замужем
21445,1,гражданский брак
21446,0,женат / замужем


In [29]:
#Сгруппируем данные по семейному положению и посчитаем количество клиентов
family_dict_groupby = family_dict.groupby('family_status').count()
family_dict_groupby

,family_status_id
family_status,
в разводе,1195
вдовец / вдова,959
гражданский брак,4149
женат / замужем,12336
не женат / не замужем,2809


Создадим словарь по образованию:

In [30]:
education_dict = df[['education_id', 'education']] 
education_dict.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


In [31]:
#В словаре большое количество дубликатов.  Удалим их.
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict 

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Категоризируем данные по количеству детей:

In [32]:
# Посчитаем сколько детей определенного возраста имеется у клиентов
main['children'].value_counts()

 0     14087
 1      4807
 2      2051
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [33]:
#Уалим строки из таблицы с -1 и 20 детьми.
main = main.loc[(main['children'] != -1) & (main['children'] != 20)]
main['children'].sort_values().value_counts()

0    14087
1     4807
2     2051
3      330
4       41
5        9
Name: children, dtype: int64

In [34]:
#Функция разбивает клиентов на категории: нет детей, есть дети
def children_group(children):
    if children == 0:
        return 'нет детей'
    return 'есть дети'

df['children_group'] = df['children'].apply(children_group) #применяем функцию children_group к столбцам 'children'
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,children_group
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,есть дети
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,есть дети
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,нет детей
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,есть дети
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38,нет детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21443,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,12,есть дети
21444,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,39,нет детей
21445,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,5,есть дети
21446,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,8,есть дети


In [35]:
# Словарь по детям
children_dict = df[['children', 'children_group']] 
children_dict

,children,children_group
0,1,есть дети
1,1,есть дети
2,0,нет детей
3,3,есть дети
4,0,нет детей
...,...,...
21443,1,есть дети
21444,0,нет детей
21445,1,есть дети
21446,3,есть дети


In [36]:
#Группируем по категориям, считаем количество клиентов в каждой категории
children_dict_groupby = children_dict.groupby('children_group').count()
children_dict_groupby

,children
children_group,
есть дети,7361
нет детей,14087


Категоризируем данные по уровню дохода:

In [37]:
#Посмотрим max, min, медианное и среднее значения, чтобы понять, на какие категории лучше разделить доход
print('Максимальный доход:', main['total_income'].max())
print('Минимальный доход:', main['total_income'].min())
print('Медиана:', main['total_income'].median())
print('Средний доход:', main['total_income'].mean())

Максимальный доход: 2265604
Минимальный доход: 20667
Медиана: 151904.0
Средний доход: 167435.05252051583


In [38]:
#Функция разбивает клиентов на категории по доходу: 
def total_income_group(total_income):
    if  20667 < total_income < 80000:
        return 'низкий доход'
    elif  80000 < total_income < 120000:
        return 'доход ниже среднего'
    elif  120000 < total_income < 200000:
        return 'средний уровень дохода'
    return 'доход выше среднего'

df['total_income_group'] = df['total_income'].apply(total_income_group)
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,children_group,total_income_group
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,есть дети,доход выше среднего
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,есть дети,доход ниже среднего
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,нет детей,средний уровень дохода
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,есть дети,доход выше среднего
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38,нет детей,средний уровень дохода
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21443,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,12,есть дети,доход выше среднего
21444,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,39,нет детей,средний уровень дохода
21445,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,5,есть дети,доход ниже среднего
21446,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,8,есть дети,доход выше среднего


In [39]:
# Словарь по доходам
total_income_dict = df[['total_income', 'total_income_group']] 
total_income_dict

,total_income,total_income_group
0,253875,доход выше среднего
1,112080,доход ниже среднего
2,145885,средний уровень дохода
3,267628,доход выше среднего
4,158616,средний уровень дохода
...,...,...
21443,224791,доход выше среднего
21444,155999,средний уровень дохода
21445,89672,доход ниже среднего
21446,244093,доход выше среднего


In [40]:
#Группируем по категориям, считаем количество клиентов в каждой категории
total_income_dict_groupby = total_income_dict.groupby('total_income_group').count()
total_income_dict_groupby

,total_income
total_income_group,
доход выше среднего,5566
доход ниже среднего,4568
низкий доход,2273
средний уровень дохода,9041


Категоризируем данные по цели получения кредита:

In [41]:
# Функция, которая возвращает новое значение столбца purpose:
def purpose_new_name(purpose):
    purpose_lemma=''.join(m.lemmatize(purpose)).strip('\n')
    if 'свадьба' in purpose_lemma:
        return 'на проведение свадьбы'
    elif 'жилье' in purpose_lemma:
        return 'операции с жильем'
    elif 'недвижимость' in purpose_lemma:
        return 'операции с недвижимостью'
    elif 'автомобиль' in purpose_lemma:
        return 'сделки по авто'
    elif 'образование' in purpose_lemma:
        return 'на образование'
    return False

df['purpose_group'] = df['purpose'].apply(purpose_new_name)#запишем новые значения в новый столбец purpose
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,children_group,total_income_group,purpose_group
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,есть дети,доход выше среднего,операции с жильем
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,есть дети,доход ниже среднего,сделки по авто
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,нет детей,средний уровень дохода,операции с жильем
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,есть дети,доход выше среднего,на образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38,нет детей,средний уровень дохода,на проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21443,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,12,есть дети,доход выше среднего,операции с жильем
21444,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,39,нет детей,средний уровень дохода,сделки по авто
21445,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,5,есть дети,доход ниже среднего,операции с недвижимостью
21446,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,8,есть дети,доход выше среднего,сделки по авто


In [42]:
purpose_group = df['purpose'].value_counts()
purpose_group

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья для сдачи                   650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    634
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              606
покупка жилой недвижимости                605
на покупку своего автомобиля              505
заняться высшим образованием      

In [43]:
# Собираем словарь по целям и считаем количество клиентов для каждой цели
purpose_dict = df[['purpose_group']]
purpose_dict = purpose_dict.groupby('purpose_group')['purpose_group'].count().to_frame()
purpose_dict.columns = ['purpose_group_count']
purpose_dict

,purpose_group_count
purpose_group,
на образование,4013
на проведение свадьбы,2323
операции с жильем,4458
операции с недвижимостью,6349
сделки по авто,4305


### Вывод

Для существующего набора данных выделено несколько словарей: 1. family_dict Нет смысла хранить в одной таблице и семейное положение и его id. По id всегда можно найти семейное положение. 2. education_dict Тоже самое.3 children_dict Распределили клиентов на категории: многодетных, имеющих 1-2 детей и клиентов, не имеющих детей вовсе. 4. total_income_dict Словарь по уровням дохода. 5. purpose_dict Словарь по целям получения дохода.Так данные стали выглядеть нагляднее. И можно приступать к анализу.

### Шаг 3. Ответьте на вопросы

In [44]:
#Вычисляем средний процент просрочек по таблице:
debt_avg= (df['debt'].sum() / df['debt'].count())
print('Средний процент просрочек по таблице: {:.2%}'.format(debt_avg))

Средний процент просрочек по таблице: 8.11%


- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [45]:
# Группируем данные по детям и вычисляем количество клиентов, у которых есть задолженность и сумму задолженности
children_debt= df.groupby('children_group')['debt'].agg(['count', 'sum'])

#Вычисляем процент людей, у которых есть задолженность.
children_debt['percent']=(children_debt['sum']/children_debt['count']).map('{:.2%}'.format)
children_debt 

,count,sum,percent
children_group,,,
есть дети,7361,676,9.18%
нет детей,14087,1063,7.55%


### Вывод

У клиентов, которые не имеют детей задолженность 7,55%. У клиентов, у которых есть дети, процент задолженности выше среднего - 9,18%. Значит вероятность возврата кредита в срок у клиентов без детей больше. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [46]:
#Группируем данные по семейному статусу и вычисляем количество клиентов, у которых есть задолженность и сумму задолженности
family_debt = main.groupby('family_status_id')['debt'].agg(['count', 'sum'])
#Вычисляем процент людей, у которых есть задолженность.
family_debt['percent']=(family_debt['sum']/family_debt['count']).map('{:.2%}'.format)
family_debt=family_debt.sort_values('percent')

#Соединяем все в одну таблицу:
family_debt_itog = family_dict.merge(family_debt, on = 'family_status_id',how = 'left')
#Удаляем дубликаты
family_debt_itog = family_debt_itog.drop_duplicates().reset_index(drop=True) 
family_debt_itog = family_debt_itog.sort_values(by = 'percent', ascending = True)
family_debt_itog

,family_status_id,family_status,count,sum,percent
2,2,вдовец / вдова,951,63,6.62%
3,3,в разводе,1189,84,7.06%
0,0,женат / замужем,12258,925,7.55%
1,1,гражданский брак,4132,385,9.32%
4,4,не женат / не замужем,2795,273,9.77%


### Вывод

Самый маленький процент задолженности у вдовец / вдова - 6,62% и у людей в разводе 7,06%. А вот не женатых/не замужних, а также состоящих в гражданском браке 9,77% и 9,32% соответственно. Наибольшее количество кредитов берут люди в браке. Процент задолженности - 7,55%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [47]:
# Группируем данные по уровню дохода и вычисляем количество клиентов, у которых есть задолженность и ее сумму
total_income_debt = df.groupby('total_income_group')['debt'].agg(['count', 'sum'])
#Вычисляем процент людей, у которых есть задолженность.
total_income_debt['percent']=(total_income_debt['sum']/total_income_debt['count']).map('{:.2%}'.format)
total_income_debt=total_income_debt.sort_values('percent')
total_income_debt

,count,sum,percent
total_income_group,,,
доход выше среднего,5566,389,6.99%
низкий доход,2273,171,7.52%
доход ниже среднего,4568,377,8.25%
средний уровень дохода,9041,802,8.87%


### Вывод

Клиенты со средним уровнем дохода (их большинство) имеют самый большой процент задолженности по кредитам - 8,87%. Лучше всего выплачивают кредиты клиенты с доходом выше среднего.

- Как разные цели кредита влияют на его возврат в срок?

In [49]:
#Группируем по целям получения кредита. Для каждой цели кредита вычисляем количество клиентов и сумму задолженности
purpose_debt = df.groupby('purpose_group')['debt'].agg(['count', 'sum'])

#Вычисляем процент людей, у которых есть задолженность.
purpose_debt['percent']=(purpose_debt['sum']/purpose_debt['count']).map('{:.2%}'.format)
purpose_debt=purpose_debt.sort_values('percent')
purpose_debt

,count,sum,percent
purpose_group,,,
операции с жильем,4458,307,6.89%
операции с недвижимостью,6349,474,7.47%
на проведение свадьбы,2323,186,8.01%
на образование,4013,370,9.22%
сделки по авто,4305,402,9.34%


### Вывод

Самый большой процент задолженности у клиентов, которые в целях получения кредита указали сделки по авто и на образование - 9,34% и 9,22% соответственно, что составляет выше уровня средних просрочек по кредитам. Клиенты, которые указали в целях получения кредита операции с жильем имеют самые небольшой процент задолженности - 6,89%. Следовательно, выдавать кредиты на операции по своему жилью предпочтительнее.

### Шаг 4. Общий вывод

В итоге проводимого исследования можно сказать, что поставленные задачи привели к следующим выводам:

    1. Изучив данные, были обнаружены пропуски, дубликаты и артефакты (отрицательные значения в трудовом стаже).
    2. Полученные данные были обработаны. Пропущенные значения в трудовом стаже были заменены на средние значения. Пропуски в доходе заменены на средние значения по роду деятельности. Были обнаружены дубликаты в семейном положении и образовании. Строки отличались регистром, поэтому они переведены к нижнему регистру.Цели получения кредита тоже дублируются. Для удаления дубликатов в целях получения кредита была проведена лемматизацию слов. Отрицательные значения в трудовом стаже заменены на положительные. Данные приготовили для анализа.
    3. Определена зависимость между наличием детей и возвратом кредита в срок. У клиентов, которые не имеют детей задолженность 7,55%. У клиентов, у которых есть дети, процент задолженности выше среднего - 9,18%. Учитывая то, что в анализируемых данных бездетных клиентов приблизительно в два раза больше, чем клиентов с детьми, можно сделать следующий вывод: вероятность возврата кредита в срок у клиентов без детей гораздо выше; 
    4. Определена зависимость между семейным положением и возвратом кредита в срок. Самый маленький процент задолженности у вдовец / вдова - 6,62% и у людей в разводе 7,06%. А вот не женатых/не замужних, а также состоящих в гражданском браке 9,77% и 9,32%, соответственно. Наибольшее количество кредитов берут люди в браке. Процент задолженности - 7,55%, что ниже уровня среднего процента задолженности;
    5. Определена зависимость между уровнем дохода и возвратом кредита в срок. Больше всего клиентов, которые берут кредит имеют средний уровнь дохода: от 120000 до 200000 за год. Их процент процент задолженности по кредитам самый большой - 8,87%. Лучше всего выплачивают кредиты клиенты с доходом выше среднего. 6.99% - процент задолженности;
    6. Разные цели кредита влияют на его возврат в срок. Самый большой процент задолженности у клиентов, которые в целях получения кредита указали сделки по авто и на образование - 9,34% и 9,22% соответственно, что составляет выше уровня средних просрочек по кредитам. Больше всего клиенты берут кредит на собственное жилье или др. недвижимость. Они же имеют самые небольшой процент задолженности - 6,89% и 7.47% соотыетственно. Следовательно, предпочтительнее выдавать кредиты на операции с жильем и др. недвижимостью.;
    
В целом получается, что вернет кредит банку с большей вероятностью клиент с доходом выше среднего уровня (с доходом выше 200000 за год), не имеющий детей, в разводе или вдова/вдовец, либо женатый/замужний. Клиент, который указал в цели получения кредита операции с жильем или др. недвижимостью, будет для банка предпочтительнее.
Самым ненадежным заемщиком для банка оказался клиент, у которого средний доход (120000 - 200000 в год), есть дети, но  не женатый/не замужняя, а также указал в цели получения кредита сделки с авто или на образование.